In [ ]:
!pip install langchain==0.3.14 langchain-community==0.3.14 openai==1.59.4 chromadb==0.6.2 tiktoken==0.8.0 pypdf langchain_openai==0.3.0

# 예제2.1

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI # 새로운 import 경로
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
os.environ['OPENAI_API_KEY'] ="<OpenAI API 키>" # API 키를 입력하세요.

# 예제2.2

In [ ]:
from google.colab import drive
from langchain_community.document_loaders import PyPDFLoader

drive.mount('/content/drive')   #드라이브 마운트

file_path = (
    "/content/drive/MyDrive/wikibooks/마스터파일/코드/2장 코드/이슈리포트 Vol.6 고령화시대 해결을 위한 기술개발.pdf"
)
# pdf load
loader = PyPDFLoader(file_path)  # PyPDFLoader를 통한 문서데이터 로드
pages = []
async for page in loader.alazy_load(): # 비동기로드
    pages.append(page)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pages[10]

Document(metadata={'source': '/content/drive/MyDrive/wikibooks/마스터파일/코드/2장 코드/이슈리포트 Vol.6 고령화시대 해결을 위한 기술개발.pdf', 'page': 10}, page_content='고령화시대 해결을 위한 기술개발\n- 11 -\n[ 2020년 고령친화산업 시장 규모 ] [ 고령친화 세부산업별 시장 규모 ]\n(단위 : 억원, %)구분2012년2020년CAGR(12~20)시장규모비중시장규모비중의약품37,79113.897,93713.412.6의료기기12,4384.532,4794.512.8식품64,01623.4186,34324.213.5화장품6,9452.521,6903.015.3용품16,6896.122,9073.14.0요양29,34910.7100,31613.816.6주거13,5465.014,3012.00.7여가93,03434.0262,33136.013.8합계273,809100.0728,305100.013.0* 출처:\u3000고령친화산업환경 변화 및 대응방안 - 고령친화산업 시장현황 및 수요분석, 한국보건산업진흥원, 2015. 07.\uf000 (국내수요 분석) 고령친화제품 및 서비스에 대한 가상수요 분석결과, 건강식품과 건강 서비스의 수요가 높게 나타남20)▪ 산업연구원의 통계자료에 따르면, 고령친화제품 중 건강식품에 대한 지불의사가 가장 많았으며, 다음으로는 노화방지 화장품, 패션용품 등의 순으로 나타남. 특히 연령이 높을수록 질병치료 의약품, 의료기기에 대한 지불의사가 높게 나타남 ▪ 고령친화서비스에 대한 고령자의 지불의사를 조사한 결과, 건강지원서비스에 대한 지불의사가 가장 높고 문화/여가 관련서비스, 주거 지원서비스 순으로 지불의사가 높은 것으로 나타났으며, 전반적으로는 연령이 증가함에 따라서 교통지원, 요양 서비스 등의 일상생활 지원서비스에 대한 지불의사가 증가하는 것으로 나타남◎ 고령친화산업의 트렌드 \uf000 (ICT활용 제품 증가) 

# 예제2.3

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=500, # 각 청크의 최대 길이를 500자로 설정합니다.
    chunk_overlap=40,  # 최소한의 중복만 허용하여 문맥의 연속성 유지
    length_function=len,
    separator="\n"
)
# 문서를 청크로 분할
texts = text_splitter.split_documents(pages)

#예제 2.4

In [ ]:
print("한 문장의 길이 :", len("첫 번째 문단입니다.")) # --11자

text = """첫 번째 문단입니다.
두 번째 문단입니다.
세 번째 문단입니다.
네 번째 문단입니다.
"""

# 작은 오버랩
text_splitter_small = CharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=5,  # 작은 오버랩
    separator="\n"
)

print("=== 작은 오버랩 (chunk_overlap=5) ===")
chunks = text_splitter_small.split_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"청크 {i}: {chunk}\n")

# 큰 오버랩
text_splitter_large = CharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=12,  # 큰 오버랩
    separator="\n"
)

print("=== 큰 오버랩 (chunk_overlap=12) ===")
chunks = text_splitter_large.split_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"청크 {i}: {chunk}\n")


한 문장의 길이 : 11
=== 작은 오버랩 (chunk_overlap=5) ===
청크 1: 첫 번째 문단입니다.
두 번째 문단입니다.

청크 2: 세 번째 문단입니다.
네 번째 문단입니다.

=== 큰 오버랩 (chunk_overlap=12) ===
청크 1: 첫 번째 문단입니다.
두 번째 문단입니다.

청크 2: 두 번째 문단입니다.
세 번째 문단입니다.

청크 3: 세 번째 문단입니다.
네 번째 문단입니다.



In [ ]:
text = """첫 번째 문단입니다.
두 번째 문단입니다.
세 번째 문단입니다.
네 번째 문단입니다."""

text_splitter = CharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=15,
    separator="\n"
)

chunks = text_splitter.split_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"청크 {i}: {chunk}\n")

청크 1: 첫 번째 문단입니다.
두 번째 문단입니다.
세 번째 문단입니다.
네 번째 문단입니다.



# 예제2.5


In [ ]:
# OpenAI 임베딩 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# OpenAI의 text-embedding-ada-002 모델을 사용하여 텍스트를 벡터로 변환하는 객체를 초기화합니다.

# Chroma 벡터 DB 생성
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="chroma_db_v1"  # 벡터 DB 저장 경로
)
# texts의 각 청크를 임베딩하여 Chroma DB에 저장합니다.

# 벡터 DB 저장
vectordb.persist()


<ipython-input-15-aa7f03cc2c04>:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


# 예제2.6

In [ ]:
# 디스크에서 벡터 스토어 로드
vectordb = Chroma(
    persist_directory="chroma_db_v1",
    embedding_function=embeddings
)

# 저장된 데이터 조회 및 출력
print("=== Chroma 벡터 DB에 저장된 데이터 ===")
collection = vectordb._collection
data = collection.get(include=['embeddings', 'documents', 'metadatas'])
print(f"\n1. 컬렉션 크기: {collection.count()} 문서")

# 첫 번째 문서의 임베딩 정보 출력
print("\n2. 첫 번째 문서의 임베딩 정보:")
print(f"벡터 차원: {len(data['embeddings'][0])}")
print(f"임베딩 벡터 (앞부분 5개 요소): {data['embeddings'][0][:5]}")

# 문서와 메타데이터 출력 (2개 예시)
print("\n3. 문서 및 메타데이터 (2개):")
for idx, (document, metadata) in enumerate(zip(data['documents'][20:22], data['metadatas'][20:22])):
    print(f"인덱스: {idx}, 문서: {document}, 메타데이터: {metadata}")


=== Chroma 벡터 DB에 저장된 데이터 ===

1. 컬렉션 크기: 54 문서

2. 첫 번째 문서의 임베딩 정보:
벡터 차원: 1536
임베딩 벡터 (앞부분 5개 요소): [-0.00659163 -0.02211564 -0.00149597 -0.02760287 -0.00189275]

3. 문서 및 메타데이터 (2개):
인덱스: 0, 문서: TIPA 이슈 리포트Vol. 6
- 10 -, 메타데이터: {'page': 9, 'source': '/content/drive/MyDrive/wikibooks/마스터파일/코드/2장 코드/이슈리포트 Vol.6 고령화시대 해결을 위한 기술개발.pdf'}
인덱스: 1, 문서: 2고령친화산업 동향◎ 시장규모 및 전망 (미국) 실버시장 규모 1위의 미국 시장은 2025년 약 3조 5천억 달러가 될 것으로 추정16)▪ 2030년 약 8천만 명의 베이비붐 세대(1946~1965년생) 모두가 65세 이상 고령층에 진입하여 초고령 사회에 도달할 것으로 추정되며, 이들은 자녀 세대보다 자산과 연금소득이 많아 은퇴 이후에도 높은 소비력을 보이며, 의약 및 건강관리 산업 등의 주요 수요층으로 해당 분야 산업을 이끄는 핵심 동력이 될 것이라 전망 (일본) 노인 인구비율이 가장 높은 일본은 2025년 실버시장 규모가 100조 엔(약 8천억 달러)을 넘을 것으로 전망되며, 일본의 실버시장은 일상생활 및 주거, 의료, 여행 등 다양한 분야에서 고령자의 수요를 반영한 제품 및 서비스가 활성화17)▪ 특히 생활용품 시장은 사용자의 건강상태나 연령과 상관없이 모든 사람이 사용하기 편리하도록 설계된 “유니버설 디자인”이 확대되고 있으며 고령층을 고려한 제품이 주류가 될 정도로 탄탄한 시장규모 형성 (중국) 고령인구가 가장 많은 중국의 실버시장은 2030년까지 20조 위안(약 3조 달러) 규모로 성장 예상18)▪ 중국은 역사상 3번의 베이비붐 세대가 있었으며, 2차 베이비붐 세대(1962~76년생)에 태

<ipython-input-16-eb842115a507>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


# 예제2.7

In [ ]:
retriever = vectordb.as_retriever(
    search_type="mmr",  # Maximal Marginal Relevance 검색
    search_kwargs={
        "k": 5,  # 최종 검색 문서 수
        "fetch_k": 8,  # 초기 검색 문서 수
        "lambda_mult": 0.7  # 다양성 가중치 (1에 가까울수록 다양성)
    }
)

# 프롬프트 작성
prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 문서들을 기반으로 질문에 정확하게 답변해주세요.
    다음 지침을 반드시 따라주세요:
    1. 문서의 정보만을 사용하여 답변하세요
    2. 제품명이 언급된 경우 반드시 포함해서 답변하세요
    3. 제품의 기능과 특징을 구체적으로 설명하세요
    4. 답변에 확신이 없는 경우, 그 부분을 명시적으로 언급하세요

    문맥: {context}"""),
    ("human", "{input}")
])

# GPT-4o-mini 모델을 사용
llm = ChatOpenAI(
    temperature=0,  # 출력이 일정하도록 온도 설정
    max_tokens=400,  # 최대 토큰 수
    model_name="gpt-4o-mini"  # 사용할 모델 이름
)


# 예제2.8

In [ ]:
# 단일 문서 처리 체인 생성
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

# 검색 및 응답 생성을 위한 최종 체인 생성
qa_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=document_chain
)


# 예제2.9

In [ ]:
def ask_question(question: str, qa_chain):
    # qa_chain.invoke()는 새로운 형식의 입력을 사용
    result = qa_chain.invoke({
        "input": question  # 'query' 대신 'input' 사용
    })
    print("질문:", question)
    print("\n답변:", result['answer'])  # 'result' 대신 'answer' 키 사용
    # source_documents가 있는 경우에만 출력
    if 'context' in result:
        print("\n참고 문서:")
        documents = result['context']
        for i, doc in enumerate(documents, 1):
            print(f"\n문서 {i}:")
            print(doc.page_content[:500], "...")
            if hasattr(doc, 'metadata'):
                print(f"(페이지: {doc.metadata.get('page', 'Unknown')})")
# 예시 질문
questions = [" 노인들이 일어나도록 도와주는 로봇의 제품 이름은 뭔가요? 그리고 특징을 알려주세요 "]

for question in questions:
    ask_question(question,qa_chain)
    print("\n" + "="*50 + "\n")


질문:  노인들이 일어나도록 도와주는 로봇의 제품 이름은 뭔가요? 그리고 특징을 알려주세요 

답변: 노인들이 일어나도록 도와주는 로봇의 제품 이름은 "이승 보조로봇 허그"입니다. 이 제품은 혼자 힘으로 일어나기 어려운 노인을 도와주는 기능을 가지고 있습니다. 사용자가 제품을 끌어안듯이 체중을 실어 기대면, 로봇팔이 노인을 감아 일으켜 세우는 방식으로 이동을 보조합니다. 또한, 이승 보조로봇 허그는 가정용과 시설용으로 구분되어 판매되고 있습니다.

참고 문서:

문서 1:
돌봄에프알티- 2015년 한국생산기술연구원 사내 벤처로 시작한 기업으로, 거동이 불편한 노약자들의 보행을 보조하기 위해 개발된 웨어러블 로봇 개발- 돌봄이 필요한 노인뿐만 아니라 돌봄을 제공하는 요양보호사와 간병인의 신체 부담 또한 감소
여가/사회참여스프링소프트- 치매 예방과 인지 능력 향상 목적의 기능성 게임이 탑재된 터치스크린 기반의 스마트 테이블인 ‘해피테이블’을 개발- 상호작용을 기반으로 하는 경쟁과 협동 방식으로 게임이 구성된 것이 특징이며, 시니어 사용자의 정확도나 반응 속도 등 게임 데이터를 분석해 인지 능력 이상 유무 진단, 치매 조기 발견 등이 가능 ...
(페이지: 12)

문서 2:
TIPA 이슈 리포트Vol. 6
- 14 -
Ⅲ. 고령화 기술·제품 동향◎ 신체활동/이동 보조 분야 (한국, 스마트 지팡기) 말하는 스마트 지팡기 ‘톡톡스틱’은 기존 지팡이에 IT 기술을 결합한 것으로 사고 및 넘어졌을 때, SOS전송 및 음성 도움 기능이 있으며, 위치반경 서비스·걸음 수 측정·온도측정 등 다방면 스마트 기능을 갖추고 있음 (한국, 이승 서포트 로봇) 혼자 힘으로 일어나기 어려운 노인을 도와주는 제품인 이승 보조로봇 ‘허그’는 노인이 제품을 끌어안듯이 체중을 실어 기대면 로봇팔을 감아 노인을 일으켜 세움으로써 이동을 보조하며, 가정용과 시설용으로 구분되어 판매 중임[ 톡톡스틱 ] [ 이승 보조로봇, 허그 ] ...
(페이지: 13)

문서 3:
* 출처: AI 반려 동물 

# 실험1. 리트리버의 중요성

In [ ]:
small_retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 2,        # 너무 적은 검색 결과
        "fetch_k": 3,  # 너무 적은 후보군
        "lambda_mult": 0.7
    }
)
# 체인에서 리트리버를 변경
qa_chain = create_retrieval_chain(
    retriever=small_retriever,
    combine_docs_chain=document_chain
)
# 예시 질문
questions = [" 노인들이 일어나도록 도와주는 로봇의 제품 이름은 뭔가요? 그리고 특징을 알려주세요 "]

for question in questions:
    ask_question(question,qa_chain)
    print("\n" + "="*50 + "\n")


질문:  노인들이 일어나도록 도와주는 로봇의 제품 이름은 뭔가요? 그리고 특징을 알려주세요 

답변: 노인들이 일어나도록 도와주는 로봇의 제품 이름은 '허그'입니다. 이승 보조로봇 '허그'는 혼자 힘으로 일어나기 어려운 노인을 도와주는 제품으로, 노인이 제품을 끌어안듯이 체중을 실어 기대면 로봇팔이 노인을 감아 일으켜 세우는 방식으로 이동을 보조합니다. 이 제품은 가정용과 시설용으로 구분되어 판매되고 있습니다.

참고 문서:

문서 1:
돌봄에프알티- 2015년 한국생산기술연구원 사내 벤처로 시작한 기업으로, 거동이 불편한 노약자들의 보행을 보조하기 위해 개발된 웨어러블 로봇 개발- 돌봄이 필요한 노인뿐만 아니라 돌봄을 제공하는 요양보호사와 간병인의 신체 부담 또한 감소
여가/사회참여스프링소프트- 치매 예방과 인지 능력 향상 목적의 기능성 게임이 탑재된 터치스크린 기반의 스마트 테이블인 ‘해피테이블’을 개발- 상호작용을 기반으로 하는 경쟁과 협동 방식으로 게임이 구성된 것이 특징이며, 시니어 사용자의 정확도나 반응 속도 등 게임 데이터를 분석해 인지 능력 이상 유무 진단, 치매 조기 발견 등이 가능 ...
(페이지: 12)

문서 2:
TIPA 이슈 리포트Vol. 6
- 14 -
Ⅲ. 고령화 기술·제품 동향◎ 신체활동/이동 보조 분야 (한국, 스마트 지팡기) 말하는 스마트 지팡기 ‘톡톡스틱’은 기존 지팡이에 IT 기술을 결합한 것으로 사고 및 넘어졌을 때, SOS전송 및 음성 도움 기능이 있으며, 위치반경 서비스·걸음 수 측정·온도측정 등 다방면 스마트 기능을 갖추고 있음 (한국, 이승 서포트 로봇) 혼자 힘으로 일어나기 어려운 노인을 도와주는 제품인 이승 보조로봇 ‘허그’는 노인이 제품을 끌어안듯이 체중을 실어 기대면 로봇팔을 감아 노인을 일으켜 세움으로써 이동을 보조하며, 가정용과 시설용으로 구분되어 판매 중임[ 톡톡스틱 ] [ 이승 보조로봇, 허그 ] ...
(페이지: 13)


